In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

data_dir = r'dataset'

IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
NUM_CLASSES = 15
EPOCHS = 20

transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * (IMG_HEIGHT // 8) * (IMG_WIDTH // 8), 128)
        self.fc2 = nn.Linear(128, NUM_CLASSES)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = self.pool(nn.ReLU()(self.conv3(x)))
        x = x.view(-1, 128 * (IMG_HEIGHT // 8) * (IMG_WIDTH // 8))
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=41472, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=15, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [3]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy =correct / total

    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch 1/20, Loss: 2.2713, Accuracy: 0.28%
Epoch 2/20, Loss: 2.0971, Accuracy: 0.30%
Epoch 3/20, Loss: 1.8903, Accuracy: 0.38%
Epoch 4/20, Loss: 1.7185, Accuracy: 0.42%
Epoch 5/20, Loss: 1.5603, Accuracy: 0.48%
Epoch 6/20, Loss: 1.4253, Accuracy: 0.53%
Epoch 7/20, Loss: 1.2549, Accuracy: 0.59%
Epoch 8/20, Loss: 1.1361, Accuracy: 0.63%
Epoch 9/20, Loss: 1.1073, Accuracy: 0.63%
Epoch 10/20, Loss: 1.0520, Accuracy: 0.66%
Epoch 11/20, Loss: 1.0011, Accuracy: 0.66%
Epoch 12/20, Loss: 0.9402, Accuracy: 0.69%
Epoch 13/20, Loss: 0.8352, Accuracy: 0.72%
Epoch 14/20, Loss: 0.8169, Accuracy: 0.73%
Epoch 15/20, Loss: 0.7941, Accuracy: 0.75%
Epoch 16/20, Loss: 0.7310, Accuracy: 0.77%
Epoch 17/20, Loss: 0.7278, Accuracy: 0.77%
Epoch 18/20, Loss: 0.5494, Accuracy: 0.83%
Epoch 19/20, Loss: 0.5677, Accuracy: 0.82%
Epoch 20/20, Loss: 0.5284, Accuracy: 0.83%
